# Inverse Kinematics Optimization

The previous doc explained features and how they define objectives of a constrained optimization problem. Here we show how to use this to solve IK optimization problems.

At the bottom there is more general text explaining the basic concepts.

## Demo of features in Inverse Kinematics

Let's setup a standard configuration. (Lock the window with "Always on Top".)

In [1]:
import sys
sys.path.append('../ry')

from libry import *
from numpy import *

In [2]:
K = Config()
D = K.view()
K.addFile('../rai-robotModels/pr2/pr2.g')
K.addFile('../test/kitchen.g')

For simplicity, let's add a frame that represents goals

In [3]:
K.addFrame("goal", "", "shape:sphere size:[0 0 0 .05] color:[.5 1 1]" )
K.setFrameState([1,.5,1,1,0,0,0], ["goal"])

The `positionDiff` (position difference in world coordinates) between `pr2L` (the yellow blob in the left hand) and `goal` is equal to zero:

In [17]:
X0 = K.getFrameState()
IK = K.komo_IK()
IK.addObjective(type=OT.eq, feature=FS.positionDiff, frames=['pr2L', 'goal'])
IK.optimize()
K.setFrameState( IK.getConfiguration(0) )
IK.getReport()

[{'feature': 'Transition',
  'scale': 0.1,
  'sos_value': 3.9658636036108907,
  'type': 'sos'},
 {'description': 'QuaternionNorms',
  'scale': 3.0,
  'sos_value': 0.0,
  'type': 'sos'},
 {'eq_sumOfAbs': 2.1966421762886768e-05,
  'feature': 'posDiff',
  'o1': 'pr2L',
  'o2': 'goal',
  'scale': 10.0,
  'type': 'eq'}]

The `positionDiff` between `pr2R` and `goal` is equal to zero:

In [8]:
K.setFrameState(X0)
IK = K.komo_IK()
IK.addObjective(type=OT.eq, feature=FS.positionDiff, frames=['pr2R', 'goal'])
IK.optimize()
K.setFrameState( IK.getConfiguration(0) )

The relative position of `goal` in `pr2R` coordinates equals [0,0,-.2] (which is 20cm straight in front of the yellow blob)

In [10]:
K.setFrameState(X0)
IK = K.komo_IK()
IK.addObjective(type=OT.eq, feature=FS.positionRel, frames=['goal','pr2R'], target=[0,0,-.2])
IK.optimize()
K.setFrameState( IK.getConfiguration(0) )

The distance between `pr2R` and `pr2L` is zero:

In [16]:
K.setFrameState(X0)
IK = K.komo_IK()
IK.addObjective(type=OT.eq, feature=FS.distance, frames=['pr2L','pr2R'])
IK.optimize()
K.setFrameState( IK.getConfiguration(0) )
IK.getReport()

[{'feature': 'Transition',
  'scale': 0.1,
  'sos_value': 0.22540705182679216,
  'type': 'sos'},
 {'description': 'QuaternionNorms',
  'scale': 3.0,
  'sos_value': 0.0,
  'type': 'sos'},
 {'eq_sumOfAbs': 0.0010594544384697202,
  'feature': 'dist',
  'o1': 'pr2L',
  'o2': 'pr2R',
  'scale': 10.0,
  'type': 'eq'}]

The 3D difference between the z-vector of `pr2R` and the z-vector of `goal`:

In [13]:
K.setFrameState(X0)
IK = K.komo_IK()
IK.addObjective(type=OT.eq, feature=FS.vectorZDiff, frames=['pr2R', 'goal'])
IK.optimize()
K.setFrameState( IK.getConfiguration(0) )

The scalar product between the z-vector of `pr2R` and the z-vector of `goal` is zero:

In [15]:
K.setFrameState(X0)
IK = K.komo_IK()
IK.addObjective(type=OT.eq, feature=FS.scalarProductZZ, frames=['pr2R', 'goal'])
IK.optimize()
K.setFrameState( IK.getConfiguration(0) )
IK.getReport()

[{'feature': 'Transition',
  'scale': 0.1,
  'sos_value': 0.025611028043548905,
  'type': 'sos'},
 {'description': 'QuaternionNorms',
  'scale': 3.0,
  'sos_value': 0.0,
  'type': 'sos'},
 {'eq_sumOfAbs': 0.0003845638147528252,
  'feature': 'vecAlign',
  'o1': 'pr2R',
  'o2': 'goal',
  'scale': 10.0,
  'type': 'eq',
  'v1': [0.0, 0.0, 1.0],
  'v2': [0.0, 0.0, 1.0]}]

etc etc

## More explanations

All methods to compute paths or configurations solve constrained optimization problems. To use them, you need to learn to define constrained optimization problems. Some definitions:

* An objective defines either a sum-of-square cost term, or an equality constraint, or an inequality constraint in the optimization problem. An objective is defined by its type and its feature. The type can be `sos` (sum-of-squares), `eq`, or `ineq`, referring to the three cases.
* A feature is a (differentiable mapping) from the decision variable (the full path, or robot configuration) to a feature space. If the feature space is, e.g., 3-dimensional, this defines 3 sum-of-squares terms, or 3 inequality, or 3 equality constraints, one for each dimension. For instance, the feature can be the 3-dim robot hand position in the 15th time slice of a path optimization problem. If you put an equality constraint on this feature, then this adds 3 equality constraints to the overall path optimization problem.
* A feature is defined by the keyword for the feature map (e.g., `pos` for position), typically by a set of frame names that tell which objects we refer to (e.g., `pr2L` for the left hand of the pr2), optionally some modifiers (e.g., a scale or target, which linearly transform the feature map), and the set of configuration IDs or time slices the feature is to be computed from (e.g., `confs=[15]` if the feat is to be computed from the 15th time slice in a path optimization problem).
* In path optimization problems, we often want to add objectives for a whole time interval rather than for a single time slice or specific configuration. E.g., avoid collisions from start to end. When adding objectives to the optimization problem we can specify whole intervals, with `times=[1., 2.]`, so that the objective is added to each configuration in this time interval.
* Some features, especially velocity and acceleration, refer to a tuple of (consecutive) configurations. E.g., when you impose an acceleration feature, you need to specify `confs=[13, 14, 15]`. Or if you use `times=[1.,1.]`, the acceleration features is computed from the configuration that corresponds to time=1 and the two configurations *before*.
* All kinematic feature maps (that depend on only one configuration) can be modified to become a velocity or acceleration features. E.g., the position feature map can be modified to become a velocity or acceleration feature.
* The `sos`, `eq`, and `ineq` always refer to the feature map to be *zero*, e.g., constraining all features to be equal to zero with `eq`. This is natural for many features, esp. when they refer to differences (e.g. `posDiff` or `posRel`, which compute the relative position between two frames). However, when one aims to constrain the feature to a non-zero constant value, one can modify the objective with a `target` specification.
* Finally, all features can be rescaled with a `scale` specification. Rescaling changes the costs that arise from `sos` objectives. Rescaling also has significant impact on the convergence behavior for `eq` and `ineq` constraints. As a guide: scale constraints so that if they *would* be treated as squared penalties (squaredPenalty optim mode, to be made accessible) convergence to reasonable approximate solutions is efficient. Then the AugLag will also converge efficiently to precise constraints.